In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import *
nuevos_datos = [
    Row(InvoiceNo="536365", StockCode="85123A", Quantity=10, UnitPrice=2.55, Description="WHITE HANGING HEART", InvoiceDate="2010-12-01 08:26:00", CustomerID=17850, Country="UNITED KINGDOM"),
    Row(InvoiceNo="999999", StockCode="99999X", Quantity=1, UnitPrice=9.99, Description="NEW PRODUCT", InvoiceDate="2011-01-01 10:00:00", CustomerID=99999, Country="FRANCE")
]

df_actualizacion = spark.createDataFrame(nuevos_datos)
df_actualizacion.write.mode("append").saveAsTable("sales")

In [0]:
df_actualizacion.write.format("delta").mode("overwrite").save("/Volumes/workspace/default/retail_lakehouse/temp_actualizacion")

In [0]:
df_fuente = spark.read.format("delta").load("/Volumes/workspace/default/retail_lakehouse/temp_actualizacion")
df_fuente.createOrReplaceTempView("temp_actualizacion")

In [0]:
df_silver = spark.read.format("delta").load("/Volumes/workspace/default/retail_lakehouse/silver")
df_silver.createOrReplaceTempView("silver_online_retail")

In [0]:
%sql
MERGE INTO silver_online_retail AS destino
USING temp_actualizacion AS fuente
ON destino.InvoiceNo = fuente.InvoiceNo AND destino.StockCode = fuente.StockCode
WHEN MATCHED THEN
  UPDATE SET
    destino.Quantity = fuente.Quantity,
    destino.UnitPrice = fuente.UnitPrice,
    destino.Description = fuente.Description,
    destino.InvoiceDate = fuente.InvoiceDate,
    destino.CustomerID = fuente.CustomerID,
    destino.Country = fuente.Country
WHEN NOT MATCHED THEN
  INSERT (InvoiceNo, StockCode, Quantity, UnitPrice, Description, InvoiceDate, CustomerID, Country)
  VALUES (fuente.InvoiceNo, fuente.StockCode, fuente.Quantity, fuente.UnitPrice, fuente.Description, fuente.InvoiceDate, fuente.CustomerID, fuente.Country)